# Лабораторная работа №2
## Обработка данных с использованием Kafka
### Consumer #2 (группа из двух обработчиков)

**Выполнил:** Васильев А.С.
**Группа:** 6131-010402D
**Преподаватель:** Попов С.Б.  
**Дата:** осень 2025

## Импорт необходимых модулей

In [4]:
from kafka import KafkaConsumer

## Конфигурация Kafka

In [5]:
BOOTSTRAP_SERVERS = ['localhost:9092'] 
TOPIC_2 = 'rides2'  # Имя топика с двумя разделами
GROUP_ID = 'rides2'

## Реализация обработчика данных (Consumer #2 в группе)

In [6]:
# Создание Consumer с указанием group_id
consumer = KafkaConsumer(
    TOPIC_2,
    bootstrap_servers=BOOTSTRAP_SERVERS,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id=GROUP_ID,
    value_deserializer=lambda x: x.decode('utf-8'),
    key_deserializer=lambda x: x.decode('utf-8') if x else None
)

print("========================================")
print("CONSUMER #2 запущен")
print(f"Топик: {TOPIC_2}")
print("Группа: rides2")
print("========================================\n")

# Словарь для хранения статистики по каждому VendorID
stats = {}
message_count = 0

print(f"{'Partition':<10} {'Offset':<10} {'VendorID':<10} {'Passengers':<12} {'Distance':<12} {'Amount':<12}")
print("-" * 75)

try:
    for message in consumer:
        # Получаем информацию о разделе и смещении
        partition = message.partition
        offset = message.offset
        
        # Парсинг данных
        row = message.value.split(',')
        
        vendor_id = row[0]
        passenger_count = int(row[3])
        trip_distance = float(row[4])
        total_amount = float(row[16])
        
        # Вывод информации о полученном сообщении
        print(f"{partition:<10} {offset:<10} {vendor_id:<10} {passenger_count:<12} {trip_distance:<12.2f} {total_amount:<12.2f}")
        
        # Инициализация статистики для нового VendorID
        if vendor_id not in stats:
            stats[vendor_id] = {
                'trip_count': 0,
                'total_passengers': 0,
                'total_distance': 0.0,
                'total_amount': 0.0
            }
        
        # Обновление статистики
        stats[vendor_id]['trip_count'] += 1
        stats[vendor_id]['total_passengers'] += passenger_count
        stats[vendor_id]['total_distance'] += trip_distance
        stats[vendor_id]['total_amount'] += total_amount
        
        message_count += 1
        
        # В реальной системе consumer работал бы постоянно
        # Для демонстрации ограничим количество сообщений

        # Остановиться после обработки всех сообщений
        #if message_count >= 87:
        #    break
        
except KeyboardInterrupt:
    print("\n\nОстановка Consumer #2...")
finally:
    consumer.close()

# Вывод результатов
print("\n" + "=" * 75)
print("РЕЗУЛЬТАТЫ ОБРАБОТКИ CONSUMER #2")
print("=" * 75 + "\n")

for vendor_id in sorted(stats.keys()):
    print(f"VendorID: {vendor_id}")
    print(f"  Количество поездок: {stats[vendor_id]['trip_count']}")
    print(f"  Всего пассажиров: {stats[vendor_id]['total_passengers']}")
    print(f"  Общее расстояние: {stats[vendor_id]['total_distance']:.2f} миль")
    print(f"  Общая сумма оплаты: ${stats[vendor_id]['total_amount']:.2f}")
    print()

print(f"Всего обработано сообщений: {message_count}")
print("\nConsumer #2 завершил работу")

CONSUMER #2 запущен
Топик: rides2
Группа: rides2

Partition  Offset     VendorID   Passengers   Distance     Amount      
---------------------------------------------------------------------------
1          0          1          1            1.50         9.30        
1          1          1          1            9.50         27.80       
1          2          1          1            9.70         33.80       
1          3          1          2            8.00         28.30       
1          4          1          1            2.80         5.80        
1          5          1          2            13.50        39.80       
1          6          1          2            6.70         30.80       
1          7          1          2            1.70         11.30       
1          8          1          1            1.10         10.80       
1          9          1          1            1.10         8.80        
1          10         1          1            3.50         15.80       
1         